In [ ]:
!pip install flair

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.datasets import ColumnCorpus
# load the trained model
model = SequenceTagger.load('/content/drive/MyDrive/colab/best-model.pt')

# Specify the path to your custom corpus
data_folder = "/content/drive/MyDrive/colab/"

# Specify the column format of your corpus (in this example, 'text' and 'ner' columns)
columns = {0: 'text', 1: 'ner'}

corpus = ColumnCorpus(data_folder, columns, train_file='training2.txt', dev_file='dev2.txt', test_file='test2.txt')

tagger: SequenceTagger = SequenceTagger.load('ner')
result = tagger.evaluate(corpus.test, gold_label_type='ner')
print(result.detailed_results)

In [ ]:
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# Specify the path to your custom corpus
data_folder = "/content/drive/MyDrive/colab"

# Specify the column format of your corpus (in this example, 'text' and 'ner' columns)
columns = {0: 'text', 1: 'ner'}

# Load your corpus
corpus = ColumnCorpus(data_folder, columns, train_file='training2.txt', dev_file='dev2.txt', test_file='test2.txt')
'''
print(len(corpus.train))
print(corpus.train[0].to_tagged_string('ner'))
'''
# Create the word embeddings
embedding_types = [
    WordEmbeddings('glove'),
    WordEmbeddings('crawl'),
    FlairEmbeddings('en-forward'),
    #FlairEmbeddings('en-backward')
]
embeddings = StackedEmbeddings(embedding_types)

# Create the tagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=corpus.make_label_dictionary(label_type='ner'),
                                       tag_type='ner',
                                       use_crf=True)
print(tagger)

# Train the tagger
trainer = ModelTrainer(tagger, corpus)
trainer.train('/content/drive/MyDrive/colab', learning_rate=0.1, mini_batch_size=32, max_epochs=150)
